## Import package

In [1]:
import pandas as pd
import os
import numpy as np
from collections import defaultdict
from tqdm import tqdm

pd.options.mode.chained_assignment = None

In [2]:
paths = sorted(os.listdir('./data/MIMIC'))
remove_paths = []
for path in paths:
    if path[-4:] != '.csv':
        remove_paths.append(path)
for path in remove_paths:
    paths.remove(path)
paths.remove('CALLOUT.csv')
paths.remove('CAREGIVERS.csv')
paths.remove('CPTEVENTS.csv')
paths.remove('DATETIMEEVENTS.csv')
paths.remove('SERVICES.csv',)
paths.remove('TRANSFERS.csv')
paths.remove('PROCEDURES_ICD.csv')
paths.remove('PROCEDUREEVENTS_MV.csv')
paths.remove('MICROBIOLOGYEVENTS.csv')
paths.remove('D_CPT.csv')
paths.remove('INPUTEVENTS_CV_Filtered1.csv')
paths.remove('INPUTEVENTS_CV_Filtered2.csv')
paths.remove('INPUTEVENTS_CV_Filtered3.csv')
paths.remove('INPUTEVENTS_CV_FilteredPDA.csv')
paths.remove('OUTPUTEVENTS_Filtered.csv')
paths.remove('OUTPUTEVENTS_FilteredPDA.csv')
paths.remove('LABEVENTS_Filtered.csv')
paths.remove('LABEVENTS_FilteredPDA.csv')
paths

['ADMISSIONS.csv',
 'CHARTEVENTS.csv',
 'DIAGNOSES_ICD.csv',
 'DRGCODES.csv',
 'D_ICD_DIAGNOSES.csv',
 'D_ICD_PROCEDURES.csv',
 'D_ITEMS.csv',
 'D_LABITEMS.csv',
 'ICUSTAYS.csv',
 'INPUTEVENTS_CV.csv',
 'INPUTEVENTS_MV.csv',
 'LABEVENTS.csv',
 'OUTPUTEVENTS.csv',
 'PATIENTS.csv',
 'PRESCRIPTIONS.csv']

## Load Data

In [3]:
datas = []
for path in tqdm(paths):
    datas.append(pd.read_csv('./data/MIMIC/' + path))

  0%|                                                                                                                                                                                          | 0/15 [00:00<?, ?it/s]/var/folders/r3/tfk14xq149s3ncn1ws04wscr0000gn/T/ipykernel_1801/2755860072.py:3: DtypeWarning: Columns (10,12,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  datas.append(pd.read_csv('./data/MIMIC/' + path))
 13%|███████████████████████▋                                                                                                                                                          | 2/15 [00:00<00:04,  3.19it/s]/var/folders/r3/tfk14xq149s3ncn1ws04wscr0000gn/T/ipykernel_1801/2755860072.py:3: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  datas.append(pd.read_csv('./data/MIMIC/' + path))
 87%|██████████████████████████████████████████████████████████████████████████████████████████

## Clean Data to remain SEPSIS patient and sort Data

In [6]:
i = paths.index('D_ICD_DIAGNOSES.csv')
D_ICD_DIAGNOSES = datas[i].copy()
D_ICD_DIAGNOSES.drop(D_ICD_DIAGNOSES[['sepsis' not in s.lower() for s in D_ICD_DIAGNOSES['LONG_TITLE']]].index, inplace=True)
D_ICD_DIAGNOSES.to_csv('./data/MIMIC/' + paths[i])
D_ICD_DIAGNOSES

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,9049,9049,9049,9050,77181,NB septicemia [sepsis],Septicemia [sepsis] of newborn
1,10304,10304,10304,11403,99591,Sepsis,Sepsis
2,10305,10305,10305,11404,99592,Severe sepsis,Severe sepsis
3,13293,13293,13293,13564,67020,Puerperal sepsis-unsp,"Puerperal sepsis, unspecified as to episode of..."
4,13294,13294,13294,13565,67022,Puerprl sepsis-del w p/p,"Puerperal sepsis, delivered, with mention of p..."
5,13295,13295,13295,13566,67024,Puerperl sepsis-postpart,"Puerperal sepsis, postpartum condition or comp..."


In [24]:
i = paths.index('DIAGNOSES_ICD.csv')
datas[i].sort_values(by=['SUBJECT_ID', 'SEQ_NUM'], inplace=True)
DIAGNOSES_ICD = datas[i].copy()
'''
drop_index = []
for j in tqdm(DIAGNOSES_ICD.index):
    for code in D_ICD_DIAGNOSES['ICD9_CODE']:
        if str(DIAGNOSES_ICD.loc[j]['ICD9_CODE']) == code:
            break
    else:
        drop_index.append(j)
DIAGNOSES_ICD.drop(drop_index, inplace=True)
del drop_index
DIAGNOSES_ICD.to_csv('./data/MIMIC/' + paths[i])
'''
DIAGNOSES_ICD

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,747,140,21,111970,11.0,99592
1,914,307,38,185910,3.0,99592
2,1084,477,61,189535,8.0,99591
3,1097,490,62,116009,4.0,99591
4,276,505,64,172056,3.0,99591
...,...,...,...,...,...,...
5404,648640,650341,99836,101018,8.0,99592
5405,649024,650470,99865,189346,4.0,99592
5406,649529,650720,99912,189380,8.0,99592
5407,648171,650990,99985,176670,7.0,99592


In [25]:
dataset = DIAGNOSES_ICD[['SUBJECT_ID', 'HADM_ID']].copy()
dataset.sort_values(by=['SUBJECT_ID', 'HADM_ID'], inplace=True)
dataset.index = range(len(dataset))
dataset

,SUBJECT_ID,HADM_ID
0,21,111970
1,38,185910
2,61,189535
3,62,116009
4,64,172056
...,...,...
5404,99836,101018
5405,99865,189346
5406,99912,189380
5407,99985,176670


In [7]:
'''
dict1 = defaultdict(list)
for i in range(len(dataset)):
    dict1[dataset['SUBJECT_ID'][i]].append(dataset['HADM_ID'][i])

for i in tqdm(range(len(datas))):
    if i == paths.index('DIAGNOSES_ICD.csv'):
        continue
    if 'SUBJECT_ID' in datas[i].columns and 'HADM_ID' in datas[i].columns:
        drop_index = []
        for j in datas[i].index:
            if datas[i]['HADM_ID'][j] not in dict1[datas[i]['SUBJECT_ID'][j]]:
                drop_index.append(j)
        datas[i].drop(drop_index, inplace=True)
        del drop_index
        datas[i].sort_values(by=['SUBJECT_ID', 'HADM_ID'], inplace=True)
    elif 'SUBJECT_ID' in datas[i].columns:
        datas[i] = datas[i][[id in dataset['SUBJECT_ID'].values for id in datas[i]['SUBJECT_ID']]]
        datas[i].sort_values(by=['SUBJECT_ID'], inplace=True)
    for column in datas[i].columns:
        if 'TIME' in column or 'DATE' in column or 'DOB' == column or 'DOD' == column or 'DOD_HOSP' == column or 'DOD_SSN' == column:
            datas[i][column] = datas[i][column].apply(lambda x : pd.Timestamp(x))
    datas[i].to_csv('./data/MIMIC/' + paths[i])
del dict1

        
i = paths.index('D_ICD_PROCEDURES.csv')
datas[i].sort_values(by=['ICD9_CODE'], inplace=True)
datas[i].to_csv('./data/MIMIC/' + paths[i])

i = paths.index('D_ITEMS.csv')
datas[i].sort_values(by=['ITEMID'], inplace=True)
datas[i].to_csv('./data/MIMIC/' + paths[i])

i = paths.index('D_LABITEMS.csv')
datas[i].sort_values(by=['ITEMID'], inplace=True)
datas[i].to_csv('./data/MIMIC/' + paths[i])
'''
for i in tqdm(range(len(datas))):
    for column in datas[i].columns:
        if 'TIME' in column or 'DATE' in column or 'DOB' == column or 'DOD' == column or 'DOD_HOSP' == column or 'DOD_SSN' == column:
            datas[i][column] = datas[i][column].apply(lambda x : pd.Timestamp(x))

100%|███████████████████████████████████████████| 15/15 [00:24<00:00,  1.64s/it]


# Look Data characteristics

In [4]:
i = paths.index('ICUSTAYS.csv')
ICUSTAYS = datas[i].copy()
ICUSTAYS.head()

,Unnamed: 0.1,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,0,385,21,21,111970,216859,carevue,MICU,MICU,52,52,2135-01-30 20:53:34,2135-02-08 05:38:46,8.3647
1,1,405,41,38,185910,248910,carevue,CCU,TSICU,7,23,2166-08-10 00:29:36,2166-09-04 13:39:23,25.5485
2,2,427,63,61,189535,217135,carevue,MICU,MICU,52,52,2119-01-20 15:58:00,2119-01-22 16:11:10,2.0091
3,3,428,64,62,116009,216609,carevue,CCU,CCU,7,7,2113-02-15 00:20:44,2113-02-17 20:09:46,2.8257
4,4,430,66,64,172056,232593,carevue,MICU,MICU,52,52,2143-03-03 09:26:21,2143-03-05 17:14:25,2.3250


In [5]:
i = paths.index('PATIENTS.csv')
PATIENTS = datas[i].copy()
PATIENTS.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,239,18,21,M,2047-04-04,2135-02-08,2135-02-08,2135-02-08,1
1,255,34,38,M,2090-08-31,NaN,NaN,NaN,0
2,275,54,61,M,2063-10-21,2119-02-03,2119-02-03,2119-02-03,1
3,276,55,62,M,2044-05-08,NaN,NaN,NaN,0
4,278,57,64,F,2116-06-27,NaN,NaN,NaN,0


In [6]:
i = paths.index('ADMISSIONS.csv')
ADMISSIONS = datas[i].copy()
ADMISSIONS.drop(ADMISSIONS[ADMISSIONS['HAS_CHARTEVENTS_DATA'] == 0].index, inplace=True)
ADMISSIONS.head()

,Unnamed: 0.1,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,...,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,0,230,20,21,111970,2135-01-30 20:50:00,2135-02-08 02:08:00,2135-02-08 02:08:00,EMERGENCY,EMERGENCY ROOM ADMIT,...,Medicare,NaN,JEWISH,MARRIED,WHITE,2135-01-30 18:46:00,2135-01-30 22:05:00,SEPSIS,1,1
1,1,19,40,38,185910,2166-08-10 00:28:00,2166-09-04 11:30:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,...,Medicare,NaN,CATHOLIC,WIDOWED,WHITE,NaN,NaN,ACUTE MYOCARDIAL INFARCTION-SEPSIS,0,1
2,2,63,61,61,189535,2119-01-04 18:12:00,2119-02-03 01:35:00,2119-02-03 01:35:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,...,Private,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,1,1
3,3,64,62,62,116009,2113-02-15 00:19:00,2113-02-19 15:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,...,Medicare,NaN,NOT SPECIFIED,MARRIED,PATIENT DECLINED TO ANSWER,2113-02-14 19:55:00,2113-02-15 01:17:00,"SEPSIS,URINARY TRACT INFECTION",0,1
4,4,66,64,64,172056,2143-03-03 09:25:00,2143-03-18 13:01:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,...,Medicaid,ENGL,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,2143-03-03 03:46:00,2143-03-03 11:00:00,FUNGAL MENINGITIS,0,1


In [7]:
i = paths.index('INPUTEVENTS_CV.csv')
INPUTEVENTS_CV = datas[i].copy()
INPUTEVENTS_CV.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,ORDERID,LINKORDERID,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE
0,112673,111797,21,111970.0,216859.0,2135-01-31 10:30:00,30056,120.0,ml,NaN,...,328907,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
1,112674,111798,21,111970.0,216859.0,2135-01-31 18:15:00,30056,120.0,ml,NaN,...,2063480,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
2,112675,111799,21,111970.0,216859.0,2135-02-01 08:00:00,30056,120.0,ml,NaN,...,3204863,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
3,112676,111800,21,111970.0,216859.0,2135-02-02 21:00:00,30056,20.0,ml,NaN,...,3659956,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
4,112677,111801,21,111970.0,216859.0,2135-02-03 21:00:00,30056,50.0,ml,NaN,...,8121802,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN


In [8]:
INPUTEVENTS_CV.columns

Index(['Unnamed: 0', 'ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID',
       'CHARTTIME', 'ITEMID', 'AMOUNT', 'AMOUNTUOM', 'RATE', 'RATEUOM',
       'STORETIME', 'CGID', 'ORDERID', 'LINKORDERID', 'STOPPED', 'NEWBOTTLE',
       'ORIGINALAMOUNT', 'ORIGINALAMOUNTUOM', 'ORIGINALROUTE', 'ORIGINALRATE',
       'ORIGINALRATEUOM', 'ORIGINALSITE'],
      dtype='object')

In [9]:
i = paths.index('INPUTEVENTS_CV.csv')
INPUTEVENTS_MV = datas[i].copy()
INPUTEVENTS_MV.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,ORDERID,LINKORDERID,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE
0,112673,111797,21,111970.0,216859.0,2135-01-31 10:30:00,30056,120.0,ml,NaN,...,328907,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
1,112674,111798,21,111970.0,216859.0,2135-01-31 18:15:00,30056,120.0,ml,NaN,...,2063480,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
2,112675,111799,21,111970.0,216859.0,2135-02-01 08:00:00,30056,120.0,ml,NaN,...,3204863,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
3,112676,111800,21,111970.0,216859.0,2135-02-02 21:00:00,30056,20.0,ml,NaN,...,3659956,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
4,112677,111801,21,111970.0,216859.0,2135-02-03 21:00:00,30056,50.0,ml,NaN,...,8121802,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN


In [10]:
INPUTEVENTS_MV.columns

Index(['Unnamed: 0', 'ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID',
       'CHARTTIME', 'ITEMID', 'AMOUNT', 'AMOUNTUOM', 'RATE', 'RATEUOM',
       'STORETIME', 'CGID', 'ORDERID', 'LINKORDERID', 'STOPPED', 'NEWBOTTLE',
       'ORIGINALAMOUNT', 'ORIGINALAMOUNTUOM', 'ORIGINALROUTE', 'ORIGINALRATE',
       'ORIGINALRATEUOM', 'ORIGINALSITE'],
      dtype='object')

In [11]:
i = paths.index('OUTPUTEVENTS.csv')
OUTPUTEVENTS = datas[i].copy()
OUTPUTEVENTS.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR
0,1572072,1675124,21,111970.0,216859.0,2135-02-02 04:00:00,40055,0.0,ml,2135-02-02 04:26:00,14206,NaN,NaN,NaN
1,1572073,1675125,21,111970.0,216859.0,2135-02-02 05:00:00,40055,5.0,ml,2135-02-02 05:10:00,14206,NaN,NaN,NaN
2,1572074,1675126,21,111970.0,216859.0,2135-02-02 06:00:00,40055,3.0,ml,2135-02-02 05:46:00,14206,NaN,NaN,NaN
3,1572075,1675127,21,111970.0,216859.0,2135-02-02 08:00:00,40055,22.0,ml,2135-02-02 07:52:00,20834,NaN,NaN,NaN
4,1572076,1675128,21,111970.0,216859.0,2135-02-02 09:00:00,40055,0.0,ml,2135-02-02 09:14:00,20834,NaN,NaN,NaN


In [12]:
i = paths.index('D_ITEMS.csv')
D_ITEMS = datas[i].copy()
D_ITEMS.head()

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
0,290,290,27,1,% Inspir. Time,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1,291,291,28,2,ABI (L),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2,292,292,29,3,ABI (R),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3,293,293,30,4,ABI Ankle BP [Right],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4,294,294,31,5,ABI Brachial BP [Right],NaN,carevue,chartevents,NaN,NaN,NaN,NaN


In [13]:
for i in range(len(D_ITEMS)):
    if D_ITEMS.iloc[i]['ITEMID'] == 30025:
        print(D_ITEMS.iloc[i])

Unnamed: 0.1              5782
Unnamed: 0                5782
ROW_ID                    5352
ITEMID                   30025
LABEL                  Heparin
ABBREVIATION               NaN
DBSOURCE               carevue
LINKSTO         inputevents_cv
CATEGORY                   NaN
UNITNAME                   NaN
PARAM_TYPE                 NaN
CONCEPTID                  NaN
Name: 5056, dtype: object


In [14]:
i = paths.index('D_LABITEMS.csv')
D_LABITEMS = datas[i].copy()
D_LABITEMS.head()

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
0,127,127,1,50800,SPECIMEN TYPE,BLOOD,BLOOD GAS,NaN
1,128,128,2,50801,Alveolar-arterial Gradient,Blood,Blood Gas,19991-9
2,129,129,3,50802,Base Excess,Blood,Blood Gas,11555-0
3,130,130,4,50803,"Calculated Bicarbonate, Whole Blood",Blood,Blood Gas,1959-6
4,131,131,5,50804,Calculated Total CO2,Blood,Blood Gas,34728-6


In [15]:
i = paths.index('DRGCODES.csv')
DRGCODES = datas[i].copy()
DRGCODES.head()

,Unnamed: 0.1,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,DRG_TYPE,DRG_CODE,DESCRIPTION,DRG_SEVERITY,DRG_MORTALITY
0,0,26583,25699,21,111970,HCFA,416,SEPTICEMIA AGE >17,NaN,NaN
1,1,26584,25700,21,111970,APR,7204,Septicemia & Disseminated Infections,4.0,4.0
2,2,25591,22272,38,185910,HCFA,541,ECMO OR TRACHEOSTOMY WITH MECHANICAL VENTILATI...,NaN,NaN
3,3,25592,22273,38,185910,APR,54,Tracheostomy W Long Term Mechanical Ventilatio...,4.0,4.0
4,4,21252,16347,61,189535,HCFA,403,LYMPHOMA & NON-ACUTE LEUKEMIA WITH COMPLICATIO...,NaN,NaN


In [116]:
i = paths.index('LABEVENTS.csv')
LABEVENTS = datas[i].copy()
LABEVENTS.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,10085,10440,21,111970.0,51279,2135-02-03 05:04:00,2.81,2.81,m/uL,abnormal
1,10086,10441,21,111970.0,51301,2135-02-03 05:04:00,26.8,26.80,K/uL,abnormal
2,10087,10442,21,111970.0,50800,2135-02-03 05:13:00,MIX,NaN,NaN,NaN
3,10088,10443,21,111970.0,50813,2135-02-03 05:13:00,2.6,2.60,mmol/L,abnormal
4,10089,10444,21,111970.0,50817,2135-02-03 05:13:00,56,56.00,%,NaN


In [17]:
i = paths.index('PRESCRIPTIONS.csv')
PRESCRIPTIONS = datas[i].copy()
PRESCRIPTIONS.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,435,1390475,21,111970,216859.0,2135-01-31,2135-01-31,BASE,NS (Mini Bag Plus),NaN,NaN,NS/MBP100I,001210,338055318.0,100mL Bag,100,ml,1,BAG,IV
1,436,1389751,21,111970,216859.0,2135-01-31,2135-01-31,MAIN,Levothyroxine Sodium,Levothyroxine Sodium,Levothyroxine Sodium,LEVO50,006649,74455211.0,50mcg Tablet,50,mcg,1,TAB,PO
2,437,1379358,21,111970,216859.0,2135-01-31,2135-01-31,MAIN,Meropenem,NaN,NaN,MERO500I,026488,310032520.0,500mg Vial,500,mg,1,VIAL,IV
3,438,1390474,21,111970,216859.0,2135-01-31,2135-02-01,BASE,SW,NaN,NaN,KCLBASE2,NaN,0.0,50ml Bag,100,ml,100,ml,IV
4,439,1390472,21,111970,216859.0,2135-01-31,2135-02-01,BASE,Iso-Osmotic Dextrose,NaN,NaN,VANCOBASE,NaN,0.0,200ml Bag,200,ml,200,ml,IV


# Features

In [22]:
# last row don't understand meaning
features = ['Shock_Index', 'elixhauser', 'SIRS', 'gender', 're_admission', 'GCS', 'SOFA', 'age', 'Albumin',
           'Arterial_pH', 'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'SGPT', 
           'Arterial_blood_gas', 'BUN', 'HCO3', 'INR', 'Arterial_lactate', 'CO2', 'Creatinine', 
           'Ionised_Ca', 'PT', 'Platelets_count', 'SGOT', 'Total_bili', 'WBC_count', 'DiaBP', 'SysBP', 
           'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 'Weight_kg', 'HR', 'SpO2'
           'input_total_tev', 'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc', 'mechvent']

binary_features = ['gender', 're_admission', 'mechvent']
norm_features = ['Shock_Index', 'elixhauser', 'SIRS', 'GCS', 'SOFA', 'age', 'Albumin', 'Arterial_pH', 
                'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'Arterial_blood_gas', 
                'HCO3', 'Arterial_lactate', 'CO2', 'Ionised_Ca', 'PT', 'Platelets_count', 'WBC_count', 
                'DiaBP', 'SysBP', 'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 
                'Weight_kg', 'HR', ]
log_features = ['SGPT', 'BUN', 'INR', 'Creatinine', 'SGOT', 'Total_bili', 'SpO2', 'input_total_tev',
                'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc']

## Add gender and age to dataset

In [26]:
dataset['DOB'] = pd.Series(['NaN'] * len(dataset))
dataset['Gender'] = pd.Series(['NaN'] * len(dataset))

for i in PATIENTS.index:
    subject_id = PATIENTS['SUBJECT_ID'].loc[i]
    gender = PATIENTS['GENDER'].loc[i]
    dob = PATIENTS['DOB'].loc[i]
    index = dataset[dataset['SUBJECT_ID'] == subject_id].index
    dataset['DOB'].loc[index] = dob
    dataset['Gender'].loc[index] = gender

dataset

,SUBJECT_ID,HADM_ID,DOB,Gender
0,21,111970,2047-04-04 00:00:00,M
1,38,185910,2090-08-31 00:00:00,M
2,61,189535,2063-10-21 00:00:00,M
3,62,116009,2044-05-08 00:00:00,M
4,64,172056,2116-06-27 00:00:00,F
...,...,...,...,...
5404,99836,101018,2064-04-13 00:00:00,F
5405,99865,189346,2107-06-25 00:00:00,F
5406,99912,189380,2068-09-26 00:00:00,M
5407,99985,176670,2127-04-08 00:00:00,M


## Add death time to dataset

In [27]:
dataset['DEATHTIME'] = pd.Series(['NaN'] * len(dataset))

for i in ADMISSIONS.index:
    subject_id = ADMISSIONS['SUBJECT_ID'].loc[i]
    hadm_id = ADMISSIONS['HADM_ID'].loc[i]
    death_time = ADMISSIONS['DEATHTIME'].loc[i]
    index = dataset[(dataset['SUBJECT_ID'] == subject_id) & (dataset['HADM_ID'] == hadm_id)].index
    dataset['DEATHTIME'].loc[index] = death_time   
    
dataset

,SUBJECT_ID,HADM_ID,DOB,Gender,DEATHTIME
0,21,111970,2047-04-04 00:00:00,M,2135-02-08 02:08:00
1,38,185910,2090-08-31 00:00:00,M,NaT
2,61,189535,2063-10-21 00:00:00,M,2119-02-03 01:35:00
3,62,116009,2044-05-08 00:00:00,M,NaT
4,64,172056,2116-06-27 00:00:00,F,NaT
...,...,...,...,...,...
5404,99836,101018,2064-04-13 00:00:00,F,NaT
5405,99865,189346,2107-06-25 00:00:00,F,2184-12-30 00:01:00
5406,99912,189380,2068-09-26 00:00:00,M,NaT
5407,99985,176670,2127-04-08 00:00:00,M,NaT


## Add ICUSTAY_ID to dataset

In [28]:
dataset['ICUSTAY_ID'] = pd.Series(['NaN'] * len(dataset))
dataset['Age'] = pd.Series(['NaN'] * len(dataset))
dataset['INTIME'] = pd.Series(['NaN'] * len(dataset))
dataset['OUTTIME'] = pd.Series(['NaN'] * len(dataset))

# use intime - dob to calculate age
for i in ICUSTAYS.index:
    subject_id = ICUSTAYS['SUBJECT_ID'].loc[i]
    hadm_id = ICUSTAYS['HADM_ID'].loc[i]
    icustay_id = ICUSTAYS['ICUSTAY_ID'].loc[i]
    intime = ICUSTAYS['INTIME'].loc[i]
    outtime = ICUSTAYS['OUTTIME'].loc[i]
    index = dataset[(dataset['SUBJECT_ID'] == subject_id) & (dataset['HADM_ID'] == hadm_id)].index
    if dataset['ICUSTAY_ID'].loc[index[0]] == 'NaN':
        dataset['ICUSTAY_ID'].loc[index[0]] = icustay_id
        dataset['INTIME'].loc[index[0]] = intime
        dataset['OUTTIME'].loc[index[0]] = outtime
        dataset['Age'].loc[index[0]] = (intime.to_pydatetime() - dataset['DOB'].loc[index[0]].to_pydatetime()).days / 365
    else:
        dataset.loc[len(dataset)] = [dataset['SUBJECT_ID'].loc[index[0]],
                                     dataset['HADM_ID'].loc[index[0]],
                                     dataset['DOB'].loc[index[0]],
                                     dataset['Gender'].loc[index[0]],
                                     dataset['DEATHTIME'].loc[index[0]],
                                     icustay_id,
                                     (intime.to_pydatetime() - dataset['DOB'].loc[index[0]].to_pydatetime()).days / 365,
                                     intime,
                                     outtime]

dataset.sort_values(by=['SUBJECT_ID', 'INTIME'])
dataset = dataset[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'INTIME', 'OUTTIME', 'Gender', 'Age', 'DEATHTIME']]
dataset

/tmp/ipykernel_3314/459815132.py:30: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  dataset.sort_values(by=['SUBJECT_ID', 'INTIME'])


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,INTIME,OUTTIME,Gender,Age,DEATHTIME
0,21,111970,216859,2135-01-30 20:53:34,2135-02-08 05:38:46,M,87.882192,2135-02-08 02:08:00
1,38,185910,248910,2166-08-10 00:29:36,2166-09-04 13:39:23,M,75.991781,NaT
2,61,189535,217135,2119-01-20 15:58:00,2119-01-22 16:11:10,M,55.284932,2119-02-03 01:35:00
3,62,116009,216609,2113-02-15 00:20:44,2113-02-17 20:09:46,M,68.819178,NaT
4,64,172056,232593,2143-03-03 09:26:21,2143-03-05 17:14:25,F,26.69863,NaT
...,...,...,...,...,...,...,...,...
6209,99333,136029,208940,2125-02-28 02:25:23,2125-02-28 11:30:16,M,300.246575,2125-02-28 09:15:00
6210,99383,176477,201433,2131-07-20 03:33:48,2131-08-10 13:45:33,M,28.350685,NaT
6211,99836,101018,200347,2116-06-05 15:09:08,2116-06-21 17:48:02,F,52.178082,NaT
6212,99836,101018,240303,2116-06-25 22:28:30,2116-07-15 18:56:37,F,52.232877,NaT


## Add re-admission to dataset

In [29]:
dataset['re_admission'] = pd.Series(['NaN'] * len(dataset))

dataset['re_admission'].loc[0] = 0
for i in dataset.index[1:]:
    if dataset['SUBJECT_ID'].loc[i] == dataset['SUBJECT_ID'].loc[i - 1]:
        dataset['re_admission'].loc[i] = 1
    else:
        dataset['re_admission'].loc[i] = 0

dataset

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,INTIME,OUTTIME,Gender,Age,DEATHTIME,re_admission
0,21,111970,216859,2135-01-30 20:53:34,2135-02-08 05:38:46,M,87.882192,2135-02-08 02:08:00,0
1,38,185910,248910,2166-08-10 00:29:36,2166-09-04 13:39:23,M,75.991781,NaT,0
2,61,189535,217135,2119-01-20 15:58:00,2119-01-22 16:11:10,M,55.284932,2119-02-03 01:35:00,0
3,62,116009,216609,2113-02-15 00:20:44,2113-02-17 20:09:46,M,68.819178,NaT,0
4,64,172056,232593,2143-03-03 09:26:21,2143-03-05 17:14:25,F,26.69863,NaT,0
...,...,...,...,...,...,...,...,...,...
6209,99333,136029,208940,2125-02-28 02:25:23,2125-02-28 11:30:16,M,300.246575,2125-02-28 09:15:00,0
6210,99383,176477,201433,2131-07-20 03:33:48,2131-08-10 13:45:33,M,28.350685,NaT,0
6211,99836,101018,200347,2116-06-05 15:09:08,2116-06-21 17:48:02,F,52.178082,NaT,0
6212,99836,101018,240303,2116-06-25 22:28:30,2116-07-15 18:56:37,F,52.232877,NaT,1


# Split time

In [30]:
hour_period = '01:00:00'

In [31]:
index = 0
split = pd.DataFrame(columns = dataset.columns)
for i in tqdm(dataset.index):
    intime, outtime = dataset['INTIME'].loc[i], dataset['OUTTIME'].loc[i]
    if intime == 'NaN' or 'outtime' == 'NaN':
        continue
    while outtime - intime > pd.Timedelta(hour_period):
        #['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'INTIME', 'OUTTIME', 'Gender', 'Age', 'DEATHTIME', 're_admission']
        split.loc[index] = [
            dataset['SUBJECT_ID'].loc[i],
            dataset['HADM_ID'].loc[i],
            dataset['ICUSTAY_ID'].loc[i],
            intime,
            intime + pd.Timedelta(hour_period),
            dataset['Gender'].loc[i],
            dataset['Age'].loc[i],
            dataset['DEATHTIME'].loc[i],
            dataset['re_admission'].loc[i],
        ]
        intime += pd.Timedelta(hour_period)
        index += 1
        
    split.loc[index] = [
        dataset['SUBJECT_ID'].loc[i],
        dataset['HADM_ID'].loc[i],
        dataset['ICUSTAY_ID'].loc[i],
        intime,
        outtime,
        dataset['Gender'].loc[i],
        dataset['Age'].loc[i],
        dataset['DEATHTIME'].loc[i],
        dataset['re_admission'].loc[i],
    ]
    
split

100%|█████████████████████████████████████| 6214/6214 [5:04:49<00:00,  2.94s/it]


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,INTIME,OUTTIME,Gender,Age,DEATHTIME,re_admission
0,21,111970,216859,2135-01-30 20:53:34,2135-01-30 21:53:34,M,87.882192,2135-02-08 02:08:00,0
1,21,111970,216859,2135-01-30 21:53:34,2135-01-30 22:53:34,M,87.882192,2135-02-08 02:08:00,0
2,21,111970,216859,2135-01-30 22:53:34,2135-01-30 23:53:34,M,87.882192,2135-02-08 02:08:00,0
3,21,111970,216859,2135-01-30 23:53:34,2135-01-31 00:53:34,M,87.882192,2135-02-08 02:08:00,0
4,21,111970,216859,2135-01-31 00:53:34,2135-01-31 01:53:34,M,87.882192,2135-02-08 02:08:00,0
...,...,...,...,...,...,...,...,...,...
1323653,99865,189346,259297,2184-12-30 22:14:22,2184-12-30 23:14:22,F,77.569863,2184-12-30 00:01:00,0
1323654,99865,189346,259297,2184-12-30 23:14:22,2184-12-31 00:14:22,F,77.569863,2184-12-30 00:01:00,0
1323655,99865,189346,259297,2184-12-31 00:14:22,2184-12-31 01:14:22,F,77.569863,2184-12-30 00:01:00,0
1323656,99865,189346,259297,2184-12-31 01:14:22,2184-12-31 02:14:22,F,77.569863,2184-12-30 00:01:00,0


In [32]:
split.to_csv('./data/dataset_split_1_hour.csv')

## Load dataset after split time

In [80]:
path = './data/dataset_split_4_hour.csv'
dataset = pd.read_csv(path)
dataset.head()

,Unnamed: 0,SUBJECT_ID,HADM_ID,ICUSTAY_ID,INTIME,OUTTIME,Gender,Age,DEATHTIME,re_admission
0,0,21,111970,216859,2135-01-30 20:53:34,2135-01-31 00:53:34,M,87.882192,2135-02-08 02:08:00,0
1,1,21,111970,216859,2135-01-31 00:53:34,2135-01-31 04:53:34,M,87.882192,2135-02-08 02:08:00,0
2,2,21,111970,216859,2135-01-31 04:53:34,2135-01-31 08:53:34,M,87.882192,2135-02-08 02:08:00,0
3,3,21,111970,216859,2135-01-31 08:53:34,2135-01-31 12:53:34,M,87.882192,2135-02-08 02:08:00,0
4,4,21,111970,216859,2135-01-31 12:53:34,2135-01-31 16:53:34,M,87.882192,2135-02-08 02:08:00,0


## ITEM_ID -> name

In [102]:
D_ITEMS[[id in [225625, 50893, 786, 1522] for id in D_ITEMS['ITEMID']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
679,1525,1525,706,786,Calcium (8.4-10.2),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
1111,2790,2790,2974,1522,Calcium,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
10798,10506,10506,14082,225625,Calcium non-ionized,Calcium non-ionized,metavision,chartevents,Labs,None,Numeric,NaN


In [103]:
D_LABITEMS[[id in [225625, 50893, 786, 1522] for id in D_LABITEMS['ITEMID']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
93,220,220,94,50893,"Calcium, Total",Blood,Chemistry,2000-8


## Add Albumin to dataset

In [18]:
# 50862, 772, 1521, 227456

In [19]:
D_ITEMS.drop(D_ITEMS[D_ITEMS.isnull()['LABEL']].index, inplace=True)
D_ITEMS[['albumin' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
665,1511,1511,692,772,Albumin (>3.2),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
1110,2789,2789,2973,1521,Albumin,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
1636,3130,3130,3499,2358,ALBUMIN,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2107,1576,1576,757,3066,albumin,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2582,3933,3933,2856,3727,Albumin (3.9-4.8),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
5041,4893,4893,5094,30008,Albumin 5%,NaN,carevue,inputevents_cv,NaN,NaN,NaN,NaN
5042,5772,5772,5342,30009,Albumin 25%,NaN,carevue,inputevents_cv,NaN,NaN,NaN,NaN
5192,4462,4462,5158,30181,Serum Albumin 5%,NaN,carevue,inputevents_cv,NaN,NaN,NaN,NaN
5465,4421,4421,5720,40548,ALBUMIN,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
6516,6674,6674,7903,42832,albumin 12.5%,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN


In [20]:
D_LABITEMS[['albumin' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
35,162,162,36,50835,"Albumin, Ascites",Ascites,Chemistry,1749-1
62,189,189,63,50862,Albumin,Blood,Chemistry,1751-7
210,337,337,211,51011,<Albumin>,Cerebrospinal Fluid (CSF),Chemistry,1746-7
218,345,345,219,51019,"Albumin, Joint Fluid",Joint Fluid,Chemistry,NaN
224,351,351,225,51025,"Albumin, Body Fluid",Other Body Fluid,Chemistry,1747-5
245,372,372,246,51046,"Albumin, Pleural",Pleural,Chemistry,1748-3
268,395,395,269,51069,"Albumin, Urine",Urine,Chemistry,1754-1
269,396,396,270,51070,"Albumin/Creatinine, Urine",Urine,Chemistry,14958-3
752,752,752,753,51555,SURFACTANT ALBUMIN RATIO,OTHER BODY FLUID,CHEMISTRY,NaN


## Add Arterial_pH to dataset

In [21]:
# 780, 1126, 223830, 50820

In [25]:
D_ITEMS.query("ITEMID == 780 | ITEMID == 1126 | ITEMID == 223830")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
673,1519,1519,700,780,Arterial pH,NaN,carevue,chartevents,ABG,NaN,NaN,NaN
870,119,119,1271,1126,Art.pH,NaN,carevue,chartevents,ABG,NaN,NaN,NaN
9765,12411,12411,12802,223830,PH (Arterial),PH (Arterial),metavision,chartevents,Labs,None,Numeric,NaN


## Add Calcium to dataset 

In [26]:
# 225625, 50893, 786, 1522

In [27]:
D_ITEMS[['calcium' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
679,1525,1525,706,786,Calcium (8.4-10.2),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
708,1554,1554,735,816,Ionized Calcium,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
959,1231,1231,1360,1350,ionized calcium,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1111,2790,2790,2974,1522,Calcium,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
1249,2928,2928,3112,1663,Calcium gluconate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1571,3065,3065,3434,2185,CALCIUM GLUCONATE,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2218,1687,1687,868,3219,calcium gluconate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2311,1780,1780,961,3347,Calcium Gluconate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2594,2684,2684,2868,3746,Calcium (8.8-10.8),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
2606,2696,2696,2880,3766,Ion Calcium,NaN,carevue,chartevents,ABG's,NaN,NaN,NaN


In [28]:
D_LABITEMS[['calcium' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
8,135,135,9,50808,Free Calcium,Blood,Blood Gas,1994-3
93,220,220,94,50893,"Calcium, Total",Blood,Chemistry,2000-8
228,355,355,229,51029,"Calcium, Body Fluid",Other Body Fluid,Chemistry,15155-5
265,392,392,266,51066,24 hr Calcium,Urine,Chemistry,6874-2
276,403,403,277,51077,"Calcium, Urine",Urine,Chemistry,2004-0
667,122,122,668,51468,Calcium Carbonate Crystals,Urine,Hematology,5773-7
668,123,123,669,51469,Calcium Oxalate Crystals,Urine,Hematology,5774-5
669,124,124,670,51470,Calcium Phosphate Crystals,Urine,Hematology,5775-2


## Add Glucose to dataset  

In [ ]:
# 220621, 225664, 50809, 811, 807, 50931, 226537, 1529

In [39]:
D_ITEMS[['glucose' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
699,1545,1545,726,807,Fingerstick Glucose,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
703,1549,1549,730,811,Glucose (70-105),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
926,1198,1198,1327,1310,FINGERSTICK GLUCOSE.,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1051,277,277,14,1455,fingerstick glucose,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1118,2797,2797,2981,1529,Glucose,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
1342,1971,1971,3205,1812,abg: glucose,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1625,3119,3119,3488,2338,finger stick glucose,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2390,1859,1859,1040,3447,Glucose Monitor #,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2592,2682,2682,2866,3744,Blood Glucose,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
2593,2683,2683,2867,3745,BloodGlucose,NaN,carevue,chartevents,Quick Admit,NaN,NaN,NaN


In [40]:
D_LABITEMS[['glucose' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
9,136,136,10,50809,Glucose,Blood,Blood Gas,2339-0
42,169,169,43,50842,"Glucose, Ascites",Ascites,Chemistry,2347-3
131,258,258,132,50931,Glucose,Blood,Chemistry,2345-7
213,340,340,214,51014,"Glucose, CSF",Cerebrospinal Fluid (CSF),Chemistry,2342-4
221,348,348,222,51022,"Glucose, Joint Fluid",Joint Fluid,Chemistry,2348-1
233,360,360,234,51034,"Glucose, Body Fluid",Other Body Fluid,Chemistry,2344-0
252,379,379,253,51053,"Glucose, Pleural",Pleural,Chemistry,2346-5
283,410,410,284,51084,"Glucose, Urine",Urine,Chemistry,2350-7
677,677,677,678,51478,Glucose,Urine,Hematology,5792-7
728,728,728,729,51529,Estimated Actual Glucose,BLOOD,CHEMISTRY,NaN


## Add Hemoglobin to dataset  

In [29]:
# 220228, 51222, 50811, 814

In [30]:
D_ITEMS[['hemoglobin' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
706,1552,1552,733,814,Hemoglobin,NaN,carevue,chartevents,Hematology,NaN,NaN,NaN
4669,982,982,2336,7965,methhemoglobin,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
9526,11528,11528,12742,220228,Hemoglobin,Hemoglobin,metavision,chartevents,Labs,g/dl,Numeric,NaN


In [31]:
D_LABITEMS[['hemoglobin' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
5,132,132,6,50805,Carboxyhemoglobin,Blood,Blood Gas,20563-3
11,138,138,12,50811,Hemoglobin,Blood,Blood Gas,718-7
14,141,141,15,50814,Methemoglobin,Blood,Blood Gas,2614-6
52,179,179,53,50852,% Hemoglobin A1c,Blood,Chemistry,4548-4
55,182,182,56,50855,Absolute Hemoglobin,Blood,Chemistry,718-7
411,538,538,412,51212,Fetal Hemoglobin,Blood,Hematology,4576-5
421,548,548,422,51222,Hemoglobin,Blood,Hematology,718-7
422,549,549,423,51223,Hemoglobin A2,Blood,Hematology,4552-6
423,550,550,424,51224,Hemoglobin C,Blood,Hematology,4561-7
424,551,551,425,51225,Hemoglobin F,Blood,Hematology,9749-3


## Add  Magnesium to dataset  

In [32]:
# 220635, 50960, 1532, 821

In [33]:
D_ITEMS[['magnesium' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
713,1559,1559,740,821,Magnesium (1.6-2.6),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
1121,2800,2800,2984,1532,Magnesium,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
1450,2079,2079,3313,1970,magnesium 2gm/hr,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3769,3616,3616,2539,6133,Magnesium Sulfate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
5058,5783,5783,5353,30027,Magnesium Sulfate,NaN,carevue,inputevents_cv,NaN,NaN,NaN,NaN
5492,6076,6076,5793,40645,magnesium,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
6643,6769,6769,8039,43018,magnesium 1gm/hr,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
7126,6246,6246,9086,44088,Magnesium,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
9554,11488,11488,12702,220635,Magnesium,Magnesium,metavision,chartevents,Labs,None,Numeric,NaN
9690,9555,9555,12986,222011,Magnesium Sulfate,Magnesium Sulfate,metavision,inputevents_mv,Medications,grams,Solution,NaN


In [34]:
D_LABITEMS[['magnesium' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
160,287,287,161,50960,Magnesium,Blood,Chemistry,2601-3
236,363,363,237,51037,"Magnesium, Body Fluid",Other Body Fluid,Chemistry,29365-4
287,414,414,288,51088,"Magnesium, Urine",Urine,Chemistry,2598-1


## Add  PTT-Partial Thromboplastin Time to dataset  

In [35]:
# 227466, 825, 1533, 51275

In [36]:
D_ITEMS[['ptt' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
717,1563,1563,744,825,PTT(22-35),NaN,carevue,chartevents,Coags,NaN,NaN,NaN
1122,2801,2801,2985,1533,PTT,NaN,carevue,chartevents,Coags,NaN,NaN,NaN
2625,2715,2715,2899,3796,Ptt,NaN,carevue,chartevents,Heme/Coag,NaN,NaN,NaN
9541,11475,11475,12689,220562,ZPTT,ZPTT,metavision,chartevents,Labs,None,Numeric,NaN
11619,10305,10305,14827,227466,PTT,PTT,metavision,chartevents,Labs,None,Numeric with tag,NaN


In [37]:
D_LABITEMS[['ptt' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
474,601,601,475,51275,PTT,Blood,Hematology,3173-2


## Add  Potassium to dataset 

In [38]:
# 829, 50971, 1535, 227464, 50822

In [39]:
D_ITEMS[['potassium' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
721,1567,1567,748,829,Potassium (3.5-5.3),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
1124,2803,2803,2987,1535,Potassium,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
1349,1978,1978,3212,1821,potassium phosphate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1860,3354,3354,3723,2691,URINE POTASSIUM,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1969,3463,3463,3832,2856,Potassium phosphate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2489,3840,3840,2763,3578,Potassium Chloride,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2580,3931,3931,2854,3725,ABG POTASSIUM,NaN,carevue,chartevents,ABG'S,NaN,NaN,NaN
2623,2713,2713,2897,3792,Potassium (3.5-5.3),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
2668,870,870,2224,4194,ABG Potassium,NaN,carevue,chartevents,ABG'S,NaN,NaN,NaN
2771,1311,1311,1440,4381,Potassium Phosphate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN


In [40]:
D_LABITEMS[['potassium' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
22,149,149,23,50822,"Potassium, Whole Blood",Blood,Blood Gas,6298-4
33,160,160,34,50833,Potassium,Other Body Fluid,Blood Gas,2821-7
47,174,174,48,50847,"Potassium, Ascites",Ascites,Chemistry,49789-1
171,298,298,172,50971,Potassium,Blood,Chemistry,2823-3
240,367,367,241,51041,"Potassium, Body Fluid",Other Body Fluid,Chemistry,2821-7
256,383,383,257,51057,"Potassium, Pleural",Pleural,Chemistry,NaN
263,390,390,264,51064,"Potassium, Stool",Stool,Chemistry,15202-5
296,423,423,297,51097,"Potassium, Urine",Urine,Chemistry,2828-2


## Add  SGPT-Serum Glutamic-Pyruvic Transaminase to dataset 

In [42]:
# 3802

In [43]:
D_ITEMS[['sgpt' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
2629,2719,2719,2903,3802,SGPT,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN


In [44]:
# only one id
sgpt_id = D_ITEMS[['sgpt' in str(s).lower() for s in D_ITEMS['LABEL']]]['ITEMID'].iloc[0]

## Add  Arterial blood gas to dataset 

In [51]:
D_ITEMS[['arterial' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
30,320,320,57,51,Arterial BP [Systolic],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
31,321,321,58,52,Arterial BP Mean,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
32,322,322,59,53,Arterial Pressure,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
669,1515,1515,696,776,Arterial Base Excess,NaN,carevue,chartevents,ABG,NaN,NaN,NaN
670,1516,1516,697,777,Arterial CO2(Calc),NaN,carevue,chartevents,ABG,NaN,NaN,NaN
671,1517,1517,698,778,Arterial PaCO2,NaN,carevue,chartevents,ABG,NaN,NaN,NaN
672,1518,1518,699,779,Arterial PaO2,NaN,carevue,chartevents,ABG,NaN,NaN,NaN
673,1519,1519,700,780,Arterial pH,NaN,carevue,chartevents,ABG,NaN,NaN,NaN
1045,271,271,8,1449,Arterial BP(Rad),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1757,3251,3251,3620,2529,arterial pulse flap,NaN,carevue,chartevents,NaN,NaN,NaN,NaN


In [52]:
D_LABITEMS[['arterial' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
1,128,128,2,50801,Alveolar-arterial Gradient,Blood,Blood Gas,19991-9


## Add  BUN-Blood Urea Nitrogen to dataset 

In [45]:
# 51006, 781, 1162, 225624

In [46]:
D_ITEMS[['bun' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
674,1520,1520,701,781,BUN (6-20),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
876,125,125,1277,1162,BUN,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2588,3939,3939,2862,3737,BUN (6-20),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
3527,2522,2522,1976,5876,bun,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4773,4025,4025,4025,8220,Effluent BUN,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10797,10505,10505,14081,225624,BUN,BUN,metavision,chartevents,Labs,None,Numeric,NaN
11438,9668,9668,14677,227000,BUN_ApacheIV,BUN_ApacheIV,metavision,chartevents,Scores - APACHE IV (2),None,Numeric,NaN
11439,9669,9669,14678,227001,BunScore_ApacheIV,BunScore_ApacheIV,metavision,chartevents,Scores - APACHE IV (2),None,Numeric,NaN


In [47]:
D_LABITEMS[D_LABITEMS['ITEMID'] == 51006]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
205,332,332,206,51006,Urea Nitrogen,Blood,Chemistry,3094-0


## Add  HCO3 to dataset 

In [48]:
# 50882, 227443, 50803

In [49]:
D_ITEMS[['hco3' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
704,1550,1550,731,812,HCO3,NaN,carevue,chartevents,ABG,NaN,NaN,NaN
1710,3204,3204,3573,2468,NAHCO3- MEQ/HR,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4777,4029,4029,4029,8235,naHCO3,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
5903,5967,5967,6338,41824,D5W 150meq NAHCO3,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
5956,8554,8554,10799,41983,NAHCO3,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
6170,5403,5403,7532,42337,NAHCO3- CC/HR,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
6590,6722,6722,7971,42939,1/4ns w 100meqnahco3,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
7017,6166,6166,8952,43904,D51/4NS+100MeqNaHCO3,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
7179,6291,6291,9152,44166,BICARBONATE-HCO3,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
7398,6937,6937,8313,44539,D5W W/ NAHCO3 150MEQ,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN


In [50]:
D_LABITEMS.query("ITEMID == 50882 | ITEMID == 50803")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
3,130,130,4,50803,"Calculated Bicarbonate, Whole Blood",Blood,Blood Gas,1959-6
82,209,209,83,50882,Bicarbonate,Blood,Chemistry,1963-8


## Add  INR-International Normalized Ratio to dataset 

In [55]:
D_ITEMS[['inr' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
707,1553,1553,734,815,INR (2-4 ref. range),NaN,carevue,chartevents,Coags,NaN,NaN,NaN
1119,2798,2798,2982,1530,INR,NaN,carevue,chartevents,Coags,NaN,NaN,NaN
6310,5504,5504,7669,42527,INR BLOOD PRODUCTS,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
7355,6897,6897,8258,44481,ER URINR OUTPUT,NaN,carevue,outputevents,NaN,NaN,NaN,NaN
9540,11474,11474,12688,220561,ZINR,ZINR,metavision,chartevents,Labs,None,Numeric,NaN
11620,10306,10306,14828,227467,INR,INR,metavision,chartevents,Labs,None,Numeric with tag,NaN


## Add  Arterial Lactate to dataset 

In [51]:
# 50813

In [52]:
D_LABITEMS.query("ITEMID == 50813")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
13,140,140,14,50813,Lactate,Blood,Blood Gas,32693-4


## Add  CO2 to dataset 

In [53]:
# 787

In [54]:
D_ITEMS.query("ITEMID == 787")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
680,1526,1526,707,787,Carbon Dioxide,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN


In [55]:
D_ITEMS[[' co2' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
670,1516,1516,697,777,Arterial CO2(Calc),NaN,carevue,chartevents,ABG,NaN,NaN,NaN
744,768,768,1145,857,Venous CO2(Calc),NaN,carevue,chartevents,VBG'S,NaN,NaN,NaN
1914,3408,3408,3777,2772,SJ CO2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2610,2700,2700,2884,3773,Mix Venous CO2(calc),NaN,carevue,chartevents,Mixed VBGs,NaN,NaN,NaN
2634,2724,2724,2908,3810,Total CO2,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
2644,2734,2734,2918,3830,Venous CO2,NaN,carevue,chartevents,Venous ABG,NaN,NaN,NaN
9527,11529,11529,12743,220235,Arterial CO2 Pressure,PCO2 (Arterial),metavision,chartevents,Labs,mmHg,Numeric,NaN
11136,11644,11644,14344,226062,Venous CO2 Pressure,PCO2 (Venous),metavision,chartevents,Labs,mmHg,Numeric,NaN


In [58]:
D_LABITEMS[[' co2' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
4,131,131,5,50804,Calculated Total CO2,Blood,Blood Gas,34728-6


## Add  Creatinine to dataset  

In [57]:
# 791, 1525, 220615, 50912

In [58]:
D_ITEMS[['creatinine' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
683,1529,1529,710,791,Creatinine (0-1.3),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
1114,2793,2793,2977,1525,Creatinine,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
2597,2687,2687,2871,3750,Creatinine (0-0.7),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
3465,2460,2460,1914,5811,urine creatinine,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
9550,11484,11484,12698,220615,Creatinine,Creatinine,metavision,chartevents,Labs,None,Numeric,NaN
11356,12476,12476,14512,226751,CreatinineApacheIIScore,CreatinineApacheIIScore,metavision,chartevents,Scores - APACHE II,None,Numeric,NaN
11357,12477,12477,14513,226752,CreatinineApacheIIValue,CreatinineApacheIIValue,metavision,chartevents,Scores - APACHE II,None,Numeric,NaN
11443,9673,9673,14682,227005,Creatinine_ApacheIV,Creatinine_ApacheIV,metavision,chartevents,Scores - APACHE IV (2),None,Numeric,NaN


In [59]:
D_LABITEMS[['creatinine' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
41,168,168,42,50841,"Creatinine, Ascites",Ascites,Chemistry,12191-3
112,239,239,113,50912,Creatinine,Blood,Chemistry,2160-0
220,347,347,221,51021,"Creatinine, Joint Fluid",Joint Fluid,Chemistry,14401-4
231,358,358,232,51032,"Creatinine, Body Fluid",Other Body Fluid,Chemistry,12190-5
251,378,378,252,51052,"Creatinine, Pleural",Pleural,Chemistry,14399-0
266,393,393,267,51067,24 hr Creatinine,Urine,Chemistry,2162-6
269,396,396,270,51070,"Albumin/Creatinine, Urine",Urine,Chemistry,14958-3
272,399,399,273,51073,"Amylase/Creatinine Ratio, Urine",Urine,Chemistry,34235-2
279,406,406,280,51080,Creatinine Clearance,Urine,Chemistry,33558-8
280,407,407,281,51081,"Creatinine, Serum",Urine,Chemistry,NaN


## Add  Ionised Calcium to dataset 

In [60]:
# 50808, 816, 225667

In [61]:
D_ITEMS[['ionized calcium' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
708,1554,1554,735,816,Ionized Calcium,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
959,1231,1231,1360,1350,ionized calcium,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4753,4005,4005,4005,8177,Ionized calcium,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4811,4063,4063,4063,8325,IONIZED CALCIUM,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10828,10536,10536,14112,225667,Ionized Calcium,Ionized Calcium,metavision,chartevents,Labs,None,Numeric,NaN


In [62]:
D_LABITEMS.query("ITEMID == 50808")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
8,135,135,9,50808,Free Calcium,Blood,Blood Gas,1994-3


## Add  PT-Prothrombin Time to dataset 

In [63]:
# 1286, 824, 227465, 51274

In [64]:
D_ITEMS.query("ITEMID == 1286 | ITEMID == 824 | ITEMID == 227465")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
716,1562,1562,743,824,PT(11-13.5),NaN,carevue,chartevents,Coags,NaN,NaN,NaN
925,1197,1197,1326,1286,PT,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
11618,10304,10304,14826,227465,Prothrombin time,PT,metavision,chartevents,Labs,None,Numeric with tag,NaN


In [65]:
D_LABITEMS.query("ITEMID == 51274")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
473,600,600,474,51274,PT,Blood,Hematology,5902-2


## Add Platelets Count to dataset 

In [66]:
# 51265, 828, 227457

In [67]:
D_ITEMS[['platelet' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
720,1566,1566,747,828,Platelets,NaN,carevue,chartevents,Hematology,NaN,NaN,NaN
2621,2711,2711,2895,3789,Platelet (150-440),NaN,carevue,chartevents,Heme/Coag,NaN,NaN,NaN
3880,4082,4082,4082,6256,Platelet Count,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
5039,5770,5770,5340,30006,Platelets,NaN,carevue,inputevents_cv,NaN,NaN,NaN,NaN
5127,5830,5830,5405,30105,OR Platelets,NaN,carevue,inputevents_cv,NaN,NaN,NaN,NaN
10508,8679,8679,13768,225170,Platelets,Platelets,metavision,inputevents_mv,Blood Products/Colloids,mL,Solution,NaN
10836,10544,10544,14120,225678,ZPlatelet Count,ZPlatelet Count,metavision,chartevents,Labs,None,Numeric,NaN
11219,12262,12262,14454,226369,OR Platelet Intake,OR Platelet Intake,metavision,inputevents_mv,Blood Products/Colloids,mL,Solution,NaN
11503,8743,8743,14576,227071,PACU Platelet Intake,PACU Platelet Intake,metavision,inputevents_mv,Blood Products/Colloids,mL,Solution,NaN
11610,10088,10088,14771,227457,Platelet Count,Platelet Count,metavision,chartevents,Labs,None,Numeric with tag,NaN


In [68]:
D_LABITEMS[['platelet' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
439,566,566,440,51240,Large Platelets,Blood,Hematology,34167-7
463,590,590,464,51264,Platelet Clumps,Blood,Hematology,40741-1
464,591,591,465,51265,Platelet Count,Blood,Hematology,777-3
465,592,592,466,51266,Platelet Smear,Blood,Hematology,778-1


## Add SGOT-Serum Glutamic-Oxaloacetic Transaminse to dataset  

In [69]:
# 3801

In [70]:
D_ITEMS[['sgot' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
2628,2718,2718,2902,3801,SGOT,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN


In [71]:
# only one id
sgot_id = D_ITEMS[['sgot' in str(s).lower() for s in D_ITEMS['LABEL']]]['ITEMID'].iloc[0]

## Add Total bilirubun to dataset  

In [73]:
# 848, 225690, 1538, 50885

In [74]:
D_ITEMS.query("ITEMID == 848 | ITEMID == 1538 | ITEMID == 803")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
695,1541,1541,722,803,Direct Bili (0-0.3),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
735,759,759,1136,848,Total Bili (0-1.5),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
1127,2806,2806,2990,1538,Total Bili,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN


In [75]:
D_ITEMS[['bilirubin' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
3044,2259,2259,1713,4948,Bilirubin,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10819,10527,10527,14103,225651,Direct Bilirubin,Direct Bilirubin,metavision,chartevents,Labs,None,Numeric,NaN
10845,10553,10553,14129,225690,Total Bilirubin,Total Bilirubin,metavision,chartevents,Labs,None,Numeric,NaN
11436,9666,9666,14675,226998,Bilirubin_ApacheIV,Bilirubin_ApacheIV,metavision,chartevents,Scores - APACHE IV (2),None,Numeric,NaN


In [131]:
D_LABITEMS.query("ITEMID == 50883")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
83,210,210,84,50883,"Bilirubin, Direct",Blood,Chemistry,1968-7


In [68]:
D_LABITEMS[['bilirubin, total' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
38,165,165,39,50838,"Bilirubin, Total, Ascites",Ascites,Chemistry,14422-0
85,212,212,86,50885,"Bilirubin, Total",Blood,Chemistry,1975-2
211,338,338,212,51012,"Bilirubin, Total, CSF",Cerebrospinal Fluid (CSF),Chemistry,1973-7
227,354,354,228,51028,"Bilirubin, Total, Body Fluid",Other Body Fluid,Chemistry,1974-5
248,375,375,249,51049,"Bilirubin, Total, Pleural",Pleural,Chemistry,14421-2


## Add White Blood Cell Count to dataset  

In [76]:
# 861, 1542, 220546, 51301, 51300, 1127

In [77]:
D_ITEMS.query("ITEMID == 861 | ITEMID == 1542 | ITEMID == 220546 | ITEMID == 1127")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
748,772,772,1149,861,"WBC (4-11,000)",NaN,carevue,chartevents,Hematology,NaN,NaN,NaN
871,120,120,1272,1127,"WBC (4-11,000)",NaN,carevue,chartevents,Hematology,NaN,NaN,NaN
1131,2810,2810,2994,1542,WBC,NaN,carevue,chartevents,Hematology,NaN,NaN,NaN
9538,11472,11472,12686,220546,WBC,WBC,metavision,chartevents,Labs,None,Numeric,NaN


In [78]:
D_ITEMS[['white' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
2647,2737,2737,2921,3834,WhiteBloodC 4.0-11.0,NaN,carevue,chartevents,Heme/Coag,NaN,NaN,NaN


In [79]:
D_LABITEMS.query("ITEMID == 51300")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
499,626,626,500,51300,WBC Count,Blood,Hematology,26464-8


In [80]:
D_LABITEMS[['white' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
500,627,627,501,51301,White Blood Cells,Blood,Hematology,804-5


## Add Diastolic Blood Pressure to dataset  

In [81]:
# 224643, 225310, 220180, 8555, 220051, 8368, 8441, 8440

In [82]:
D_ITEMS.query("ITEMID == 225310 | ITEMID == 8555 | ITEMID == 8368 | ITEMID == 8441 | ITEMID == 8440")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
4829,4637,4637,4831,8368,Arterial BP [Diastolic],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4901,4709,4709,4903,8440,Manual BP [Diastolic],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4902,4710,4710,4904,8441,NBP [Diastolic],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
5016,4823,4823,5017,8555,Arterial BP #2 [Diastolic],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10598,12444,12444,13688,225310,ART BP Diastolic,ART BP Diastolic,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN


In [83]:
D_ITEMS[['blood pressure' in str(s).lower() and 'diastolic' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
9501,11503,11503,12717,220051,Arterial Blood Pressure diastolic,ABPd,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN
9519,11521,11521,12735,220180,Non Invasive Blood Pressure diastolic,NBPd,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN
10150,11323,11323,13424,224643,Manual Blood Pressure Diastolic Left,Manual BPd L,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN
11545,9441,9441,14618,227242,Manual Blood Pressure Diastolic Right,Manual BPd R,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN


## Add Systolic Blood Pressure to dataset  

In [84]:
# 442, 227243, 224167, 220179, 225309

In [85]:
D_ITEMS.query("ITEMID == 442 | ITEMID == 225309")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
381,671,671,408,442,Manual BP [Systolic],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10597,12443,12443,13687,225309,ART BP Systolic,ART BP Systolic,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN


In [86]:
D_ITEMS[['blood pressure' in str(s).lower() and 'systolic' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
9500,11502,11502,12716,220050,Arterial Blood Pressure systolic,ABPs,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN
9518,11520,11520,12734,220179,Non Invasive Blood Pressure systolic,NBPs,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN
9958,9314,9314,13050,224167,Manual Blood Pressure Systolic Left,Manual BPs L,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN
11546,9442,9442,14619,227243,Manual Blood Pressure Systolic Right,Manual BPs R,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN


## Add Mean Blood Pressure to dataset  

In [88]:
# 456, 220181, 224, 225312, 220052, 52, 6702, 224322

In [89]:
D_ITEMS.query("ITEMID == 456 | ITEMID == 224 | ITEMID == 225312 | ITEMID == 52 | ITEMID == 6702 | ITEMID == 224322")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
31,321,321,58,52,Arterial BP Mean,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
195,485,485,222,224,IABP Mean,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
393,683,683,420,456,NBP Mean,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4124,5063,5063,4326,6702,Arterial BP Mean #2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10018,10164,10164,13110,224322,IABP Mean,MAP - Assisted,metavision,chartevents,IABP,mmHg,Numeric,NaN
10599,12445,12445,13689,225312,ART BP mean,ART BP mean,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN


In [90]:
D_ITEMS[['blood pressure mean' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
9502,11504,11504,12718,220052,Arterial Blood Pressure mean,ABPm,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN
9520,11522,11522,12736,220181,Non Invasive Blood Pressure mean,NBPm,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN


## Add PaCO2 (Partial pressure of carbon dioxide) to dataset  

In [91]:
# 226062, 778, 220235, 50818

In [115]:
D_ITEMS.query("ITEMID == 226062 | ITEMID == 778 | ITEMID == 220235")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
671,1517,1517,698,778,Arterial PaCO2,NaN,carevue,chartevents,ABG,NaN,NaN,NaN
9527,11529,11529,12743,220235,Arterial CO2 Pressure,PCO2 (Arterial),metavision,chartevents,Labs,mmHg,Numeric,NaN
11136,11644,11644,14344,226062,Venous CO2 Pressure,PCO2 (Venous),metavision,chartevents,Labs,mmHg,Numeric,NaN


In [93]:
D_LABITEMS.query("ITEMID == 50818")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
18,145,145,19,50818,pCO2,Blood,Blood Gas,11557-6


## Add PaO2 (partial pressure of oxygen) to dataset  

In [94]:
# 779

In [95]:
D_ITEMS[['pao2' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
423,713,713,450,490,PAO2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
672,1518,1518,699,779,Arterial PaO2,NaN,carevue,chartevents,ABG,NaN,NaN,NaN


## Add FiO2 (fractional inspired oxygen) to dataset  

In [96]:
# 189, 727, 223835

In [97]:
D_ITEMS.query("ITEMID == 223835")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
9767,12413,12413,12804,223835,Inspired O2 Fraction,FiO2,metavision,chartevents,Respiratory,None,Numeric,NaN


In [98]:
D_ITEMS[['fio2' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
159,449,449,186,185,FIO2 Alarm-High,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
160,450,450,187,186,FIO2 Alarm-Low,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
163,453,453,190,189,FiO2 (Analyzed),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
164,454,454,191,190,FiO2 Set,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
165,455,455,192,191,FiO2/O2 Delivered,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
649,1495,1495,676,727,Vision FiO2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
796,45,45,1197,1040,BIpap FIO2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
897,146,146,1298,1206,HFO FIO2:,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1377,2006,2006,3240,1863,HFO-FiO2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1749,3243,3243,3612,2518,HFO- FIO2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN


## Add PaO/FiO2 ratio to dataset  

## Add Respiratory Rate to dataset   

In [99]:
# 224422, 618, 220210, 224689, 614, 651, 224690, 615

In [100]:
D_ITEMS.query("ITEMID == 224422 | ITEMID == 614 | ITEMID == 651 | ITEMID == 615")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
544,259,259,571,614,Resp Rate (Spont),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
545,260,260,572,615,Resp Rate (Total),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
577,1423,1423,604,651,Spon RR (Mech.),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10065,10413,10413,13339,224422,Spont RR,Spont RR,metavision,chartevents,Respiratory,bpm,Numeric,NaN


In [101]:
D_ITEMS[['respiratory rate' in str(s).lower()in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
548,263,263,575,618,Respiratory Rate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
549,1395,1395,576,619,Respiratory Rate Set,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
9522,11524,11524,12738,220210,Respiratory Rate,RR,metavision,chartevents,Respiratory,insp/min,Numeric,NaN
10181,10052,10052,13244,224688,Respiratory Rate (Set),Respiratory Rate (Set),metavision,chartevents,Respiratory,insp/min,Numeric,NaN
10182,10053,10053,13245,224689,Respiratory Rate (spontaneous),Respiratory Rate (spontaneous),metavision,chartevents,Respiratory,insp/min,Numeric,NaN
10183,10054,10054,13246,224690,Respiratory Rate (Total),Respiratory Rate (Total),metavision,chartevents,Respiratory,insp/min,Numeric,NaN


## Add Temperature (Celsius) to dataset     

In [102]:
# 223761, 677, 676, 679, 678, 223762

In [103]:
D_ITEMS.query("ITEMID == 223761 | ITEMID == 678")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
602,1448,1448,629,678,Temperature F,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
9720,12366,12366,12757,223761,Temperature Fahrenheit,Temperature F,metavision,chartevents,Routine Vital Signs,?F,Numeric,NaN


In [104]:
D_ITEMS[['temperature' in str(s).lower() and 'c' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
600,1446,1446,627,676,Temperature C,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
601,1447,1447,628,677,Temperature C (calc),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
603,1449,1449,630,679,Temperature F (calc),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4998,4813,4813,5007,8537,"Temp/Iso/Warmer [Temperature, degrees C]",NaN,carevue,chartevents,NaN,NaN,NaN,NaN
9721,12367,12367,12758,223762,Temperature Celsius,Temperature C,metavision,chartevents,Routine Vital Signs,?C,Numeric,NaN
10172,10043,10043,13235,224674,Changes in Temperature,Changes in Temperature,metavision,chartevents,Toxicology,NaN,Text,NaN
11211,12254,12254,14446,226329,Blood Temperature CCO (C),Blood Temp CCO (C),metavision,chartevents,Routine Vital Signs,?C,Numeric,NaN
11492,9415,9415,14731,227054,TemperatureF_ApacheIV,TemperatureF_ApacheIV,metavision,chartevents,Scores - APACHE IV (2),?F,Numeric,NaN


## Add Weight (kg) to dataset     

In [105]:
# 226531, 763, 224639, 226512

In [106]:
D_ITEMS.query("ITEMID == 226531 | ITEMID == 763 | ITEMID == 224639")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
661,1507,1507,688,763,Daily Weight,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10146,11319,11319,13420,224639,Daily Weight,Daily Weight,metavision,chartevents,General,kg,Numeric,NaN
11259,11885,11885,14401,226531,Admission Weight (lbs.),Admission Weight (lbs.),metavision,chartevents,General,None,Numeric,NaN


In [107]:
D_ITEMS[['weight' in str(s).lower() and 'kg' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
2491,3842,3842,2765,3580,Present Weight (kg),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2494,3845,3845,2768,3583,Previous Weight (kg),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2575,3926,3926,2849,3693,Weight Kg,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2578,3929,3929,2852,3723,Birth Weight (kg),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2658,860,860,2214,4183,Birthweight (kg),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
11249,11875,11875,14391,226512,Admission Weight (Kg),Admission Weight (Kg),metavision,chartevents,General,kg,Numeric,NaN


## Add Heart Rate to dataset     

In [108]:
# 211, 220045

In [109]:
D_ITEMS[['heart rate' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
185,475,475,212,211,Heart Rate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2428,1897,1897,1078,3494,Lowest Heart Rate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
9496,11498,11498,12712,220045,Heart Rate,HR,metavision,chartevents,Routine Vital Signs,bpm,Numeric,NaN
9497,11499,11499,12713,220046,Heart rate Alarm - High,HR Alarm - High,metavision,chartevents,Alarms,bpm,Numeric,NaN
9498,11500,11500,12714,220047,Heart Rate Alarm - Low,HR Alarm - Low,metavision,chartevents,Alarms,bpm,Numeric,NaN


## Add SpO2 (Oxygen Saturation) to dataset     

In [110]:
# 646, 50817, 834, 220277

In [111]:
D_ITEMS.query("ITEMID == 646 | ITEMID == 834 | ITEMID == 220277")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
572,1418,1418,599,646,SpO2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
725,749,749,1126,834,SaO2,NaN,carevue,chartevents,ABG's,NaN,NaN,NaN
9530,12355,12355,12746,220277,O2 saturation pulseoxymetry,SpO2,metavision,chartevents,Respiratory,%,Numeric,NaN


In [112]:
D_LABITEMS.query("ITEMID == 50817")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
17,144,144,18,50817,Oxygen Saturation,Blood,Blood Gas,20564-1


## Add Fluid Output - 4 hourly period to dataset      

## Add Mechanical Ventilation to dataset      

In [113]:
D_ITEMS[['ventilation' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
10592,12438,12438,13682,225303,Mask Ventilation (Intubation),Mask Ventilation (Intubation),metavision,chartevents,Intubation,NaN,Text,NaN
10932,10213,10213,13977,225792,Invasive Ventilation,Invasive Ventilation,metavision,procedureevents_mv,2-Ventilation,None,Process,NaN
10933,10214,10214,13978,225794,Non-invasive Ventilation,Non-invasive Ventilation,metavision,procedureevents_mv,2-Ventilation,None,Process,NaN


In [114]:
D_LABITEMS[['ventilation' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
27,154,154,28,50827,Ventilation Rate,Blood,Blood Gas,NaN
